# Kickstarter Data Preparation

## Data Info

> [Kickstarter dataset](https://www.kaggle.com/c/kickstarter-success/data) from Kaggle

> For students' use to predict success of a Kickstarter project

In [1]:
import pandas as pd
import numpy as np

In [2]:
filename1 = "ks-projects-201612.csv"
df1 = pd.read_csv(filename1, encoding = "ISO-8859-1")

df1.head()

C:\Users\Kenisha.Priester\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,10/9/2015 11:36,1000,8/11/2015 12:12,0,failed,0,GB,0,NaN,NaN,NaN,NaN
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2/26/2013 0:20,45000,1/12/2013 0:20,220,failed,3,US,220,NaN,NaN,NaN,NaN
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,4/16/2012 4:24,5000,3/17/2012 3:24,1,failed,1,US,1,NaN,NaN,NaN,NaN
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,8/29/2015 1:00,19500,7/4/2015 8:35,1283,canceled,14,US,1283,NaN,NaN,NaN,NaN
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,4/1/2016 13:38,50000,2/26/2016 13:38,52375,successful,224,US,52375,NaN,NaN,NaN,NaN


***

## Identifying Messy Data

In [3]:
df1.columns

Index(['ID ', 'name ', 'category ', 'main_category ', 'currency ', 'deadline ',
       'goal ', 'launched ', 'pledged ', 'state ', 'backers ', 'country ',
       'usd pledged ', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16'],
      dtype='object')

13 out of 17 columns have whitespace at end of column name

In [4]:
#take extra whitespace off end of column names
df1.rename(columns=lambda x: x.strip(), inplace=True)

#### Find what kind of data is contained in 'Unnamed:' columns

In [5]:
df1[df1['Unnamed: 16'].notnull()]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
269970,677103185,SixSixSeven,Angels,Demons,Religion,Esoteric,Graphic Novels,Comics,USD,10/10/2015 1:00,750,9/10/2015 18:15,25,failed,1,US,25.0


Data is incorrectly placed in other cells resulting in extra column names

#### Move data into correct columns

In [6]:
#manually copying values from one cell to another
#reference column by index label
#take value from 'goal' and copy it to 'main_category'
df1.loc[269970, 'main_category'] = df1.loc[269970, 'goal']

In [7]:
#reference column by index number
#take value from 'currency' and copy it to 'category
df1.iloc[269970, 2] = df1.iloc[269970, 4]

In [8]:
#before w/ manual replacements
df1.iloc[269970]

ID                     677103185
name                SixSixSeven 
category               Religion 
main_category     Graphic Novels
currency               Religion 
deadline                Esoteric
goal              Graphic Novels
launched                  Comics
pledged                      USD
state            10/10/2015 1:00
backers                      750
country          9/10/2015 18:15
usd pledged                   25
Unnamed: 13               failed
Unnamed: 14                    1
Unnamed: 15                   US
Unnamed: 16                   25
Name: 269970, dtype: object

In [9]:
#x = row index; y = column index

#move data in 'Unnamed: 16'
def shiftData16(x): 
    n = list(range(17))
    for i in n:
        if (i < 4): pass
        if ((i >=4) & (i < 13)):
            df1.iloc[x, i] = df1.iloc[x, i+4]
        elif (i >= 13) : 
            df1.iloc[x, i] = np.NaN
    return x

Row with data in 'Unnamed: 16' column is shifted 4 spaces to right from column index 4 - 13

In [10]:
#running function on row 269970
shiftData16(269970)

269970

In [11]:
#after
df1.iloc[269970]

ID                     677103185
name                SixSixSeven 
category               Religion 
main_category     Graphic Novels
currency                     USD
deadline         10/10/2015 1:00
goal                         750
launched         9/10/2015 18:15
pledged                       25
state                     failed
backers                        1
country                       US
usd pledged                   25
Unnamed: 13                  NaN
Unnamed: 14                  NaN
Unnamed: 15                  NaN
Unnamed: 16                  NaN
Name: 269970, dtype: object

Successfully moved data into correct columns, leaving all 'Unnamed' columns null

#### Repeat for all rows that contain data in any 'Unnamed' columns

In [12]:
df_15 = df1[df1['Unnamed: 15'].notnull()]
df_15

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
104120,1618382802,Druid Hill Park Passport: Discover,Enjoy,Learn,Be active!,Publishing,Publishing,USD,7/28/2012 1:30,9500,6/6/2012 23:54,9854,successful,208,US,9854,NaN
145502,1864561559,T-shirt,dress shirts,pants,clothing company,Apparel,Fashion,USD,8/2/2015 18:23,20000,7/3/2015 18:23,0,failed,0,US,0,NaN
264243,642928449,FRANCIS - The Pope,Buenos Aires,soccer,tango and culture,Film & Video,Film & Video,USD,3/2/2014 23:34,75000,2/5/2014 2:06,0,failed,0,US,0,NaN


In [13]:
#test getting index numbers
df_15_index = df_15.index
df_15_index

Int64Index([104120, 145502, 264243], dtype='int64')

In [14]:
#function to get index numbers

def getIndex(x):
    return x.index

In [15]:
#x = row index; y = column index

#copy data in 'Unnamed: 15' columns 2 spaces to the left (exception: index (0,1) get passed; index (13:16) get NaN)
def shiftData15(x): 
    n = list(range(17))
    for i in n:
        if (i < 2): pass
        if ((i >=2) & (i < 13)):
            df1.iloc[x, i] = df1.iloc[x, i+3]
        elif (i >= 13) : 
            df1.iloc[x, i] = np.NaN
    return x

In [16]:
#Unnamed: 15
for i in getIndex(df_15):
    shiftData15(i)

In [17]:
#now it's empty
df1[df1['Unnamed: 15'].notnull()]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16


In [18]:
#test if new values are in place
df15ls = list(getIndex(df_15))

df1.iloc[df15ls]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
104120,1618382802,Druid Hill Park Passport: Discover,Publishing,Publishing,USD,7/28/2012 1:30,9500,6/6/2012 23:54,9854,successful,208,US,9854,NaN,NaN,NaN,NaN
145502,1864561559,T-shirt,Apparel,Fashion,USD,8/2/2015 18:23,20000,7/3/2015 18:23,0,failed,0,US,0,NaN,NaN,NaN,NaN
264243,642928449,FRANCIS - The Pope,Film & Video,Film & Video,USD,3/2/2014 23:34,75000,2/5/2014 2:06,0,failed,0,US,0,NaN,NaN,NaN,NaN


In [19]:
df_14 = df1[df1['Unnamed: 14'].notnull()]
df_14

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
13795,1081139420,The Rolling Stones,BEGGARS BANQUET,50 Years in the Making,Rock,Music,USD,8/12/2011 1:17,4625,6/4/2011 1:17,20,failed,2,US,20,NaN,NaN
16482,1097082409,Pixel art,personalized,Retro Gaming art.,Crafts,Crafts,USD,7/28/2016 21:36,500,6/28/2016 21:36,0,failed,0,US,0,NaN,NaN
61447,1365419849,VIA,ME (V.isual I.nteractive A.rchive,M.ercury E.dition),Software,Technology,USD,3/19/2016 20:40,10000,2/18/2016 21:40,26,failed,2,US,26,NaN,NaN
112934,1670090635,EXCALIBRE:Sword of Truth,Path of Light,Kingdom of Heaven.,Nonfiction,Publishing,USD,4/1/2016 8:00,24000,2/22/2016 21:57,1,failed,1,US,1,NaN,NaN
225692,410977509,Handmade Artisan Soaps: Natural,Gentle,Good for your skin,Crafts,Crafts,USD,10/8/2015 22:08,200,9/8/2015 22:08,435,successful,10,US,435,NaN,NaN
243349,516522480,American Booty,USA-made products Series One,Spirits,Painting,Art,USD,2/2/2013 3:45,1776,1/3/2013 3:45,401,failed,7,US,401,NaN,NaN
275186,708058934,Be Wise,Wear Wisdom,Divine Wisdom,Apparel,Fashion,USD,2/27/2015 13:53,10000,1/28/2015 13:53,1,failed,1,US,1,NaN,NaN
305503,890269949,Community Clothing. Make Clothes,Create Jobs,Restore Pride,Fashion,Fashion,GBP,3/16/2016 16:24,75000,2/15/2016 17:24,88619.2,successful,1020,GB,128535,NaN,NaN


In [20]:
#copied over 2 rows to the left
def shiftData14(x): 
    n = list(range(17))
    for i in n:
        if (i < 2): pass
        if ((i >=2) & (i < 13)):
            df1.iloc[x, i] = df1.iloc[x, i+2]
        elif (i >= 13) : 
            df1.iloc[x, i] = np.NaN
    return x

In [21]:
#Unnamed: 14
for i in getIndex(df_14):
    shiftData14(i)

In [22]:
#test for any values left in Unnamed: 14
df1[df1['Unnamed: 14'].notnull()]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16


In [23]:
#test if new values are in place
df14ls = list(getIndex(df_14))

df1.iloc[df14ls]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
13795,1081139420,The Rolling Stones,Rock,Music,USD,8/12/2011 1:17,4625,6/4/2011 1:17,20,failed,2,US,20,NaN,NaN,NaN,NaN
16482,1097082409,Pixel art,Crafts,Crafts,USD,7/28/2016 21:36,500,6/28/2016 21:36,0,failed,0,US,0,NaN,NaN,NaN,NaN
61447,1365419849,VIA,Software,Technology,USD,3/19/2016 20:40,10000,2/18/2016 21:40,26,failed,2,US,26,NaN,NaN,NaN,NaN
112934,1670090635,EXCALIBRE:Sword of Truth,Nonfiction,Publishing,USD,4/1/2016 8:00,24000,2/22/2016 21:57,1,failed,1,US,1,NaN,NaN,NaN,NaN
225692,410977509,Handmade Artisan Soaps: Natural,Crafts,Crafts,USD,10/8/2015 22:08,200,9/8/2015 22:08,435,successful,10,US,435,NaN,NaN,NaN,NaN
243349,516522480,American Booty,Painting,Art,USD,2/2/2013 3:45,1776,1/3/2013 3:45,401,failed,7,US,401,NaN,NaN,NaN,NaN
275186,708058934,Be Wise,Apparel,Fashion,USD,2/27/2015 13:53,10000,1/28/2015 13:53,1,failed,1,US,1,NaN,NaN,NaN,NaN
305503,890269949,Community Clothing. Make Clothes,Fashion,Fashion,GBP,3/16/2016 16:24,75000,2/15/2016 17:24,88619.2,successful,1020,GB,128535,NaN,NaN,NaN,NaN


In [24]:
df_13 = df1[df1['Unnamed: 13'].notnull()]
len(df_13)

613

In [25]:
df_13.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
1454,1008705746,Zephyra´s new full length,'As The World Collapses',Metal,Music,SEK,2/2/2016 0:56,15000,1/3/2016 0:56,4262,failed,14,SE,504.9476528,NaN,NaN,NaN
1563,1009317190,French Cuisine,A Traditional Experience,Cookbooks,Food,USD,9/8/2014 0:46,13730,8/9/2014 3:16,3984,failed,46,US,3984,NaN,NaN,NaN
1794,1010871699,The Beginners Guide to being Unsuicidal,the one act,Theater,Theater,USD,12/31/2011 23:25,5000,11/21/2011 23:25,525,failed,10,US,525,NaN,NaN,NaN
1931,1011687764,Best OnLine Classifieds,Ever / No More Spam,Web,Technology,USD,9/20/2014 19:56,6300,8/21/2014 19:56,0,failed,0,US,0,NaN,NaN,NaN
2420,101453314,Social Media Ruined My Life,A Short Film from Adam S Curtis,Shorts,Film & Video,USD,3/14/2013 20:11,3000,2/25/2013 21:11,3035,successful,42,US,3035,NaN,NaN,NaN


In [26]:
#copied over 1 row to the left
def shiftData13(x): 
    n = list(range(17))
    for i in n:
        if (i < 2): pass
        if ((i >=2) & (i < 13)):
            df1.iloc[x, i] = df1.iloc[x, i+1]
        elif (i >= 13) : 
            df1.iloc[x, i] = np.NaN
    return x

In [27]:
#Unnamed: 13
for i in getIndex(df_13):
    shiftData13(i)

In [28]:
df1[df1['Unnamed: 13'].notnull()]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16


In [29]:
#test if new values are in place
df13ls = list(getIndex(df_13))

df1.iloc[df13ls].head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
1454,1008705746,Zephyra´s new full length,Metal,Music,SEK,2/2/2016 0:56,15000,1/3/2016 0:56,4262,failed,14,SE,504.9476528,NaN,NaN,NaN,NaN
1563,1009317190,French Cuisine,Cookbooks,Food,USD,9/8/2014 0:46,13730,8/9/2014 3:16,3984,failed,46,US,3984,NaN,NaN,NaN,NaN
1794,1010871699,The Beginners Guide to being Unsuicidal,Theater,Theater,USD,12/31/2011 23:25,5000,11/21/2011 23:25,525,failed,10,US,525,NaN,NaN,NaN,NaN
1931,1011687764,Best OnLine Classifieds,Web,Technology,USD,9/20/2014 19:56,6300,8/21/2014 19:56,0,failed,0,US,0,NaN,NaN,NaN,NaN
2420,101453314,Social Media Ruined My Life,Shorts,Film & Video,USD,3/14/2013 20:11,3000,2/25/2013 21:11,3035,successful,42,US,3035,NaN,NaN,NaN,NaN


***

## Check Data

In [30]:
df1.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,10/9/2015 11:36,1000,8/11/2015 12:12,0,failed,0,GB,0,NaN,NaN,NaN,NaN
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2/26/2013 0:20,45000,1/12/2013 0:20,220,failed,3,US,220,NaN,NaN,NaN,NaN
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,4/16/2012 4:24,5000,3/17/2012 3:24,1,failed,1,US,1,NaN,NaN,NaN,NaN
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,8/29/2015 1:00,19500,7/4/2015 8:35,1283,canceled,14,US,1283,NaN,NaN,NaN,NaN
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,4/1/2016 13:38,50000,2/26/2016 13:38,52375,successful,224,US,52375,NaN,NaN,NaN,NaN


In [31]:
df1.count()

ID               323750
name             323746
category         323750
main_category    323750
currency         323750
deadline         323750
goal             323750
launched         323750
pledged          323750
state            323750
backers          323750
country          323750
usd pledged      319960
Unnamed: 13           0
Unnamed: 14           0
Unnamed: 15           0
Unnamed: 16           0
dtype: int64

No longer need 'Unnamed' columns because it's all null

In [32]:
#remove Unnamed columns
to_drop = ['Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15','Unnamed: 16']
df1.drop(to_drop, inplace=True, axis=1)
df1.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,10/9/2015 11:36,1000,8/11/2015 12:12,0,failed,0,GB,0
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2/26/2013 0:20,45000,1/12/2013 0:20,220,failed,3,US,220
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,4/16/2012 4:24,5000,3/17/2012 3:24,1,failed,1,US,1
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,8/29/2015 1:00,19500,7/4/2015 8:35,1283,canceled,14,US,1283
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,4/1/2016 13:38,50000,2/26/2016 13:38,52375,successful,224,US,52375


#### Check data types before converting

In [33]:
df1.dtypes

ID                int64
name             object
category         object
main_category    object
currency         object
deadline         object
goal             object
launched         object
pledged          object
state            object
backers          object
country          object
usd pledged      object
dtype: object

In [35]:
pd.to_numeric(df1['backers'])
#raised an error
#df1['backers'].astype('int64') also raises error

ValueError: Unable to parse string "undefined" at position 64484

#### Check data in 'backers' column

In [36]:
#rows where 'backers' did not convert to numeric
df_back = df1.loc[pd.to_numeric(df1['backers'], errors='coerce').isnull()]
df_back

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged
64484,1383759393,CD: Heartsong Harmonics,Marc & Mary sing together at last!,Music,Music,USD,11/3/2015 0:59,7085,9/14/2015 23:55,7470,undefined,0,"N,""0"
85619,1508366697,Everyday Beautiful,a day in the life of a champion,Film & Video,Film & Video,USD,3/27/2015 1:00,2000,2/25/2015 6:52,0,undefined,0,"N,""0"
98614,1585608231,Hana Zara's new album,The North (2015),Music,Music,USD,3/3/2015 2:04,3000,1/17/2015 2:04,3280,undefined,0,"N,""0"
123506,1733391784,Celebrating Joni Mitchell,Songs by & about her + Stories,Music,Music,CAD,10/11/2015 0:47,5000,9/11/2015 0:47,6323,undefined,0,"N,""0"
159427,194816108,Legal Highs,The Sobering Truth,Film & Video,Film & Video,GBP,11/5/2014 3:00,5000,10/16/2014 3:00,0,undefined,0,"N,""0"
163067,1969863991,Puppy Training Steps,Right and Wrong,Film & Video,Film & Video,USD,4/24/2015 22:07,5000,3/25/2015 21:07,0,undefined,0,"N,""0"
177636,2057841246,Adventure to Peru's Sacred Valley,Recording Soundtracks,Music,Music,USD,3/21/2015 21:13,2200,3/2/2015 22:14,2503,undefined,0,"N,""0"


Data is shifted 1 space to the right

In [37]:
def shiftDataBack(x): 
    n = list(range(13))
    for i in n:
        if (i < 2): pass
        if ((i >=2) & (i < 12)):
            df1.iloc[x, i] = df1.iloc[x, i+1]
        elif (i == 12) : 
            df1.iloc[x, i] = np.NaN
    return x

In [38]:
for i in getIndex(df_back):
    shiftDataBack(i)

In [39]:
df1.loc[pd.to_numeric(df1['backers'], errors='coerce').isnull()]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged


In [40]:
#test if new values are in place
dfbackls = list(getIndex(df_back))

df1.iloc[dfbackls]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged
64484,1383759393,CD: Heartsong Harmonics,Music,Music,USD,11/3/2015 0:59,7085,9/14/2015 23:55,7470,undefined,0,"N,""0",NaN
85619,1508366697,Everyday Beautiful,Film & Video,Film & Video,USD,3/27/2015 1:00,2000,2/25/2015 6:52,0,undefined,0,"N,""0",NaN
98614,1585608231,Hana Zara's new album,Music,Music,USD,3/3/2015 2:04,3000,1/17/2015 2:04,3280,undefined,0,"N,""0",NaN
123506,1733391784,Celebrating Joni Mitchell,Music,Music,CAD,10/11/2015 0:47,5000,9/11/2015 0:47,6323,undefined,0,"N,""0",NaN
159427,194816108,Legal Highs,Film & Video,Film & Video,GBP,11/5/2014 3:00,5000,10/16/2014 3:00,0,undefined,0,"N,""0",NaN
163067,1969863991,Puppy Training Steps,Film & Video,Film & Video,USD,4/24/2015 22:07,5000,3/25/2015 21:07,0,undefined,0,"N,""0",NaN
177636,2057841246,Adventure to Peru's Sacred Valley,Music,Music,USD,3/21/2015 21:13,2200,3/2/2015 22:14,2503,undefined,0,"N,""0",NaN


#### Retry of converting 'backers' column to int data type

In [41]:
df1['backers'].astype('int64')

0           0
1           3
2           1
3          14
4         224
5          16
6          40
7          58
8          43
9           0
10        100
11          0
12          0
13         11
14         16
15         20
16          7
17         40
18        624
19         12
20          3
21         66
22        147
23          0
24        571
25         27
26          1
27        549
28         18
29         30
         ... 
323720      1
323721     37
323722     42
323723     39
323724    413
323725      0
323726     97
323727     37
323728      5
323729     78
323730      5
323731    103
323732      2
323733      0
323734      0
323735    120
323736     38
323737     31
323738     16
323739      5
323740      0
323741     78
323742     36
323743      1
323744      0
323745      1
323746      5
323747      1
323748      6
323749     17
Name: backers, Length: 323750, dtype: int64

Success!

#### Convert other columns to "useful" data types

In [42]:
#change goal, pledged, and usd pledged to float values
df1[['goal', 'pledged', 'usd pledged']] = df1[['goal', 'pledged', 'usd pledged']].apply(pd.to_numeric)

In [43]:
#change 'deadline' and 'launched' to datetime type
df1[['deadline', 'launched']] = df1[['deadline', 'launched']].apply(pd.to_datetime)

In [44]:
#change ID to string type
df1['ID'] = df1['ID'].astype(str)

#### One last check before clean data file export

In [45]:
df1.dtypes

ID                       object
name                     object
category                 object
main_category            object
currency                 object
deadline         datetime64[ns]
goal                    float64
launched         datetime64[ns]
pledged                 float64
state                    object
backers                  object
country                  object
usd pledged             float64
dtype: object

In [46]:
#export reformatted data to csv file
df1.to_csv('ks-projects-201612clean.csv', index=False)

***

# Notes

- In row with data in 'Unnamed: 16' column, chose religion as 'category' and to encompass angels & demons (other cell values)
- For columns that contained "graphic novels" in one column and "comics" in another (in Kickstarter, the correct name is "Graphic Novels/Comics" ), I chose to keep "graphic novels" for ease of shifting
- Noticed that rows in 'backers' column that did not convert also had N,"0 as a country value (left that for students to make decision how to handle)